In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import types as T
from pyspark.sql import window as W
from pyspark.sql import functions as F

spark = (
    SparkSession 
    .builder 
    .appName("Streaming Process Files") 
    .config("spark.streaming.stopGracefullyOnShutdown", True) 
    .master("local[*]") 
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")

spark.conf.set("spark.sql.shuffle.partitions", 8)
spark.conf.set("spark.sql.streaming.schemaInference", True)

df_01 = spark.read.json("/opt/workspace/device_01.json")
df_01.show()

24/07/17 01:34:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+----------+--------------------+--------------------+-----------+--------------+--------------------+
|customerId|                data|             eventId|eventOffset|eventPublisher|           eventTime|
+----------+--------------------+--------------------+-----------+--------------+--------------------+
|   CI00103|[[[D001, C, ERROR...|e3cb26d3-41b2-49a...|      10001|        device|2023-01-05 11:13:...|
+----------+--------------------+--------------------+-----------+--------------+--------------------+



In [3]:
df_01 = (
    spark
    .readStream
    # .option("cleanSource", "archive") # off,delete,archive
    # .option("sourceArchiveDir", "archive/02_file_streaming") # archive되는 파일의 이동 경로
    .option("maxFilesPerTrigger", 1)
    .format("json")
    .load("/opt/workspace/")
)


stream = (
    df_01
    .writeStream
    .format("memory")
    .queryName("memory")
    .outputMode("append")
    .option("checkpointLocation", "checkpoint_dir/02_file_streaming")
    .start()
)

stream.awaitTermination()



KeyboardInterrupt: 

In [ ]:
spark.sql("SELECT * FROM memory").show()